In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
!pip install -q -U ../../

In [2]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [3]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [8]:
from scipy import interpolate

In [9]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
logging.basicConfig(level=logging.INFO)

In [12]:
dataset_name = 'covtype'

In [13]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

INFO:sdgym.data:Loading dataset covtype (all variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []


In [14]:
data.shape

(481012, 55)

In [15]:
categorical_columns, ordinal_columns

([10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54],
 [])

In [16]:
for i in range(data.shape[1]):
    print (i, len(np.unique(data[:, i])))

0 1963
1 361
2 67
3 543
4 683
5 5758
6 205
7 183
8 255
9 5796
10 2
11 2
12 2
13 2
14 2
15 2
16 2
17 2
18 2
19 2
20 2
21 2
22 2
23 2
24 2
25 2
26 2
27 2
28 2
29 2
30 2
31 2
32 2
33 2
34 2
35 2
36 2
37 2
38 2
39 2
40 2
41 2
42 2
43 2
44 2
45 2
46 2
47 2
48 2
49 2
50 2
51 2
52 2
53 2
54 7


In [17]:
data[:, 2]

array([17., 19.,  7., ..., 22., 30., 11.], dtype=float32)

# run sdgym

In [18]:
def KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns):
    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns + ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        reg_lambda=1,
        gamma=0,
        max_depth=9
    )
#     clf = MLPClassifier(alpha=0.1, random_state=0, max_iter=1000, early_stopping=True)
    
    kde = KDECopulaNNPdf(
        use_KDE=False, 
        categorical_columns=categorical_columns+ordinal_columns,
        distinct_threshold=-1,
        n_uniform_bins=25,
#        clf=clf,
#         ordering='mi',
    )
    kde = kde.fit(data)
    
#     X_gen, sample_weight = kde.sample(n_samples)
    X_gen = kde.sample_no_weights(n_samples, show_progress=True, mode='cheap')
    
    X_gen[:, categorical_columns+ordinal_columns] = np.round(X_gen[:, categorical_columns+ordinal_columns])
    X_gen = np.float32(X_gen)
    
    print(X_gen[:, :5])

    return X_gen

In [19]:
def KDECopulaNNPdf_SynthesizerInteger(real_data, categorical_columns, ordinal_columns):
    """Census has integer only..."""
    data = KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns)
    data = np.round(data)
    
    print(data[:, :5])

    return data

In [20]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer)

all_synthesizers = [
    #IdentitySynthesizer,
    #IndependentSynthesizer,
    #KDECopulaNNPdf_Synthesizer,
    KDECopulaNNPdf_SynthesizerInteger,
]

In [21]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name], iterations=1)

INFO:sdgym.benchmark:0%|          | 0/1 [00:00<?, ?it/s]
INFO:sdgym.benchmark:Evaluating KDECopulaNNPdf_SynthesizerInteger on dataset covtype; iteration 0; 541.54 MB
INFO:sdgym.data:Loading dataset covtype (all variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []
INFO:sdgym.benchmark:Running KDECopulaNNPdf_SynthesizerInteger on dataset covtype; iteration 0; 669.76 MB
INFO:KDECopulaNNPdf:Processing 10 numerical and 45 categorical columns
INFO:KDECopulaNNPdf:Transforming numerical variables.
INFO:KDECopulaNNPdf:Configuring Bayesian Network (cat+num).
INFO:KDECopulaNNPdf:Configuring classifier.
INFO:KDECopulaNNPdf:Generating 481012 data points.

Generating for node: 14:  89%|████████▉ | 49/55 [00:29<00:02,  2.14it/s]


Generating for node: 41: 100%|██████████| 55/55 [00:32<00:00,  1.67it/s]
INFO:KDECopulaNNPdf:Fitting discriminative learner: selected 55 features.
INFO:KDECopulaNNPdf:Calibrating classifier.
INFO:KDECopulaNNPdf:Maximum weight found: 752.4226693012395
INFO:KDECopulaNNPdf:Generating 481012 data points.

Generating for node: 38:  84%|████████▎ | 46/55 [00:29<00:04,  1.91it/s]


Generating for node: 41: 100%|██████████| 55/55 [00:33<00:00,  1.64it/s]
INFO:sdgym.benchmark:Scoring KDECopulaNNPdf_SynthesizerInteger on dataset covtype; iteration 0; 1.16 GB


[[ 3.1580232e+03  3.2002193e+01  1.0000221e+01  1.0800082e+02
   3.0055223e+00]
 [ 2.4674316e+03  3.5599670e+02  2.3999500e+01  2.9980564e+01
   2.0048428e+00]
 [ 2.9098657e+03  7.6000389e+01  2.6000256e+01  3.1900473e+02
   8.2941170e+01]
 ...
 [ 2.9005371e+03  1.5915596e+02  1.1001251e+01 -9.8289400e-03
   9.0633268e-03]
 [ 2.9940295e+03  5.2000042e+01  7.9995904e+00  1.7150363e+02
   4.9993029e+00]
 [ 3.2533572e+03  9.9983437e+01  1.7000500e+01  1.9200676e+02
   9.9733698e-01]]
[[ 3.158e+03  3.200e+01  1.000e+01  1.080e+02  3.000e+00]
 [ 2.467e+03  3.560e+02  2.400e+01  3.000e+01  2.000e+00]
 [ 2.910e+03  7.600e+01  2.600e+01  3.190e+02  8.300e+01]
 ...
 [ 2.901e+03  1.590e+02  1.100e+01 -0.000e+00  0.000e+00]
 [ 2.994e+03  5.200e+01  8.000e+00  1.720e+02  5.000e+00]
 [ 3.253e+03  1.000e+02  1.700e+01  1.920e+02  1.000e+00]]


INFO:sdgym.evaluate:Evaluating using multiclass classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using multiclass classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
INFO:sdgym.benchmark:Finished KDECopulaNNPdf_SynthesizerInteger on dataset covtype; iteration 0; 1.13 GB
INFO:sdgym.benchmark:100%|##########| 1/1 [10:36<00:00, 636.07s/it]


In [22]:
scores

,covtype/accuracy,covtype/macro_f1,covtype/micro_f1,timestamp
VEEGANSynthesizer,0.218520,0.095916,0.218520,2020-04-12 09:41:35.096775
CLBNSynthesizer,0.563920,0.329636,0.563920,2020-10-17 09:46:54.494331
CTGAN,0.565890,0.316620,0.565890,2020-10-17 09:46:54.494331
CTGANSynthesizer,0.581583,0.329751,0.581583,2020-10-17 09:46:54.494331
CopulaGAN,0.569887,0.326941,0.569887,2020-10-17 09:46:54.494331
GaussianCopulaCategorical,0.421560,0.140020,0.421560,2020-10-17 09:46:54.494331
GaussianCopulaCategoricalFuzzy,0.420843,0.138726,0.420843,2020-10-17 09:46:54.494331
GaussianCopulaOneHot,0.506743,0.182262,0.506743,2020-10-17 09:46:54.494331
IdentitySynthesizer,0.758887,0.652621,0.758887,2020-10-17 09:46:54.494331
IndependentSynthesizer,0.376220,0.113155,0.376220,2020-10-17 09:46:54.494331


In [23]:
scores.tail(4)

,covtype/accuracy,covtype/macro_f1,covtype/micro_f1,timestamp
TVAESynthesizer,0.654793,0.456447,0.654793,2020-10-17 09:46:54.494331
TableganSynthesizer,0.000000,0.000000,0.000000,2020-10-17 09:46:54.494331
UniformSynthesizer,0.143150,0.087674,0.143150,2020-10-17 09:46:54.494331
KDECopulaNNPdf_SynthesizerInteger,0.599870,0.424553,0.599870,2021-05-16 07:19:26.133433
